In [1]:
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import eurostat

In [2]:
# build the URL
# {host_url}/rest/data/{version}/{format}/{lang}/{datasetCode}?{filters}

api_url = "http://ec.europa.eu/eurostat/wdds/rest/data/v2.1"
api_format = "/json"
api_lang = "/en"


In [3]:
# Name of the dataset to query
api_dataset = "/ei_hppi_q"


# Create dictionary that passes all filters to apply to the query 
filters = {   'indic': 'TOTAL', 
              #'unit': 'I15_NSA',
              'time': ['2020Q3','2020Q2','2020Q1'],
              'geo':'LU',
             }


In [4]:
# Get data
try:
    response = requests.get(api_url+api_format+api_lang+api_dataset, 
                params=filters)
    response.raise_for_status()
except requests.exceptions.HTTPError as err:
    raise SystemExit(err)
    

In [5]:
# Extract JSON data from the response
data = response.json()

In [6]:
data

{'class': 'dataset',
 'dimension': {'geo': {'category': {'index': {'LU': 0},
    'label': {'LU': 'Luxembourg'}},
   'label': 'geo'},
  'indic': {'category': {'index': {'TOTAL': 0}, 'label': {'TOTAL': 'Total'}},
   'label': 'indic'},
  'time': {'category': {'index': {'2020Q1': 0, '2020Q2': 1, '2020Q3': 2},
    'label': {'2020Q1': '2020Q1', '2020Q2': '2020Q2', '2020Q3': '2020Q3'}},
   'label': 'time'},
  'unit': {'category': {'index': {'I15_NSA': 0, 'RT1': 1, 'RT4': 2},
    'label': {'I15_NSA': 'Index, 2015=100 (NSA)',
     'RT1': 'Growth rate on previous period (t/t-1)',
     'RT4': 'Growth rate on the same quarter in previous year'}},
   'label': 'unit'}},
 'extension': {'datasetId': 'ei_hppi_q',
  'description': None,
  'lang': 'EN',
  'status': {'label': {'p': 'provisional'}},
  'subTitle': None},
 'href': 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/ei_hppi_q?indic=TOTAL&time=2020Q3&time=2020Q2&time=2020Q1&geo=LU',
 'id': ['indic', 'unit', 'geo', 'time'],
 'label': 'Hou

In [7]:
# Flatten data and load to data frame, with _ separators
dimensions = pd.json_normalize(data,[["dimension", "time", "category" ,"label"]])

print(list(dimensions))
print(dimensions)

[0]
        0
0  2020Q1
1  2020Q2
2  2020Q3


In [8]:

# Flatten data and load to data frame, with _ separators
dimensions = pd.json_normalize(data,[["dimension","geo", "category" ,"index"]][0])

print(list(dimensions))
print(dimensions)

[0]
    0
0  LU


In [9]:
# Load data to a data frame
value = pd.json_normalize(data["value"])

#pd.DataFrame(data["value"])

                       # View the data's dtypes
print(value.dtypes)
print(value.head())

0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
8    float64
dtype: object
        0       1       2    3    4    5     6     7     8
0  142.67  148.74  152.69  4.0  4.3  2.7  14.1  13.2  13.6


In [10]:
value["0"]

0    142.67
Name: 0, dtype: float64

In [1]:
#----Unemployment Rate---------
dataEU_unempl=pd.read_json('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/ei_lmhr_m?geo=EA&indic=LM-UN-T-TOT&s_adj=NSA&unit=PC_ACT',typ='series',orient='table') #,typ='DataFrame',orient='table'
x=[]
for i in sorted(int(v) for v in dataEU_unempl['value'].keys()):
    x=np.append(x,dataEU_unempl['value'][str(i)])
EU_unempl=pd.Series(x,index=pd.date_range((pd.to_datetime((sorted(dataEU_unempl['dimension']['time']['category']['index'].keys())[(sorted(int(v) for v in dataEU_unempl['value'].keys())[0])]),format='%YM%M')), periods=len(x), freq='M')) #'1/1993'

EU_unempl

NameError: name 'pd' is not defined

In [12]:
#----Money market interest rates---------
dataEU_intRates=pd.read_json('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/irt_st_m?geo=EA&intrt=MAT_ON',typ='series',orient='table') #,typ='DataFrame',orient='table'
x=[]
for i in sorted(int(v) for v in dataEU_intRates['value'].keys()):
    x=np.append(x,dataEU_intRates['value'][str(i)])
EU_intRates=pd.Series(x,index=pd.date_range((pd.to_datetime((sorted(dataEU_intRates['dimension']['time']['category']['index'].keys())[(sorted(int(v) for v in dataEU_intRates['value'].keys())[0])]),format='%YM%M')), periods=len(x), freq='M'))

EU_intRates

1994-01-31 00:01:00    6.84
1994-02-28 00:01:00    6.73
1994-03-31 00:01:00    6.68
1994-04-30 00:01:00    6.22
1994-05-31 00:01:00    5.85
                       ... 
2132-08-31 00:01:00   -0.43
2132-09-30 00:01:00   -0.46
2132-10-31 00:01:00   -0.49
2132-11-30 00:01:00   -0.51
2132-12-31 00:01:00   -0.52
Freq: M, Length: 1668, dtype: float64

In [13]:
#----GDP---------
dataEU_GDP=pd.read_json('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/namq_10_gdp?geo=EA&na_item=B1GQ&s_adj=NSA&unit=CP_MEUR',typ='series',orient='table') #,typ='DataFrame',orient='table'

x=[]
for i in sorted(int(v) for v in dataEU_GDP['value'].keys()):
    x=np.append(x,dataEU_GDP['value'][str(i)])
EU_GDP=pd.Series(x,index=pd.date_range((pd.Timestamp(sorted(dataEU_GDP['dimension']['time']['category']['index'].keys())[(sorted(int(v) for v in dataEU_GDP['value'].keys())[0])])), periods=len(x), freq='Q'))
EU_GDP

1995-03-31    1341393.4
1995-06-30    1385948.7
1995-09-30    1386328.5
1995-12-31    1470766.6
1996-03-31    1402318.4
                ...    
2019-09-30    2978123.6
2019-12-31    3093872.9
2020-03-31    2859756.1
2020-06-30    2589734.4
2020-09-30    2874899.3
Freq: Q-DEC, Length: 103, dtype: float64

In [14]:
#---- ---------
#dataEU_immo=pd.read_json('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/ei_hppi_q?geo=LU&indic=TOTAL',typ='series',orient='table') #,typ='DataFrame',orient='table'

#x=[]
#for i in sorted(int(v) for v in dataEU_immo['value'].keys()):
#    x=np.append(x,dataEU_immo['value'][str(i)])

#dataEU_immo=pd.Series(x,index=pd.date_range((pd.to_datetime((sorted(dataEU_immo['dimension']['time']['category']['index'].keys())[(sorted(int(v) for v in dataEU_immo['value'].keys())[0])]),format='%YM%M')), periods=len(x), freq='M')) #'1/1993'

#dataEU_immo

In [15]:
df_eurostat = eurostat.get_data_df('ei_hppi_q')

In [16]:
df_eurostat

,indic,unit,geo\time,2005Q1,2005Q2,2005Q3,2005Q4,2006Q1,2006Q2,2006Q3,...,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,2020Q2,2020Q3
0,TOTAL,I15_NSA,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,118.51,120.37,122.18,122.38,126.40,127.81,129.81,131.85,135.02,139.23
1,TOTAL,I15_NSA,BE,69.68,71.24,73.78,75.24,76.70,78.79,81.07,...,108.73,110.93,110.34,111.54,112.01,115.96,115.61,115.49,117.04,119.56
2,TOTAL,I15_NSA,BG,76.01,79.26,81.55,83.52,87.47,88.91,92.96,...,124.03,124.99,126.15,129.45,130.26,131.97,133.99,135.54,134.07,138.33
3,TOTAL,I15_NSA,CY,93.41,95.38,98.79,100.64,103.04,106.44,110.64,...,104.01,103.12,107.04,107.93,112.73,105.64,106.51,109.13,109.48,104.21
4,TOTAL,I15_NSA,CZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,128.70,131.90,134.40,137.20,140.60,143.40,146.40,149.00,151.50,155.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,TOTAL,RT4,SE,NaN,NaN,NaN,NaN,14.00,13.50,12.10,...,-1.70,-2.10,0.70,1.50,2.20,2.90,3.30,4.50,3.30,3.70
103,TOTAL,RT4,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.70,9.40,8.30,7.60,7.40,7.10,4.80,4.70,5.20,3.30
104,TOTAL,RT4,SK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.00,4.40,6.80,5.70,8.30,11.50,10.90,13.10,9.70,8.50
105,TOTAL,RT4,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.80,6.30,4.50,3.20,1.70,6.80,9.90,15.00,25.70,27.40


## Use the Eurostat module

In [17]:
toc_df=eurostat.get_toc_df()
toc_df

,title,code,type,last update of data,last table structure change,data start,data end
0,Database by themes,data,folder,,,,
1,General and regional statistics,general,folder,,,,
2,European and national indicators for short-ter...,euroind,folder,,,,
3,Business and consumer surveys (source: DG ECFIN),ei_bcs,folder,,,,
4,Consumer surveys (source: DG ECFIN),ei_bcs_cs,folder,,,,
...,...,...,...,...,...,...,...
10205,Enterprises that provided training to develop/...,isoc_ske_ittn2,dataset,21.01.2021,08.02.2021,2012,2020
10206,Participation in education and training - cont...,sks_devcvt,folder,,,,
10207,Enterprises providing training by type of trai...,trng_cvt_01s,dataset,26.02.2019,08.02.2021,2005,2015
10208,Participants in CVT courses by sex and size cl...,trng_cvt_12s,dataset,26.02.2019,08.02.2021,2005,2015


In [25]:
eurostat.subset_toc_df(toc_df, 'unemploy')

,title,code,type,last update of data,last table structure change,data start,data end
48,Harmonised unemployment (1 000) - monthly data,ei_lmhu_m,dataset,08.02.2021,08.02.2021,1983M01,2020M12
49,Harmonised unemployment rates (%) - monthly data,ei_lmhr_m,dataset,08.02.2021,08.02.2021,1983M01,2020M12
372,Regional unemployment - LFS annual series,lfst_r_lfu,folder,,,,
373,"Unemployment by sex, age and NUTS 2 regions (1...",lfst_r_lfu3pers,dataset,11.11.2020,08.02.2021,1999,2019
374,"Unemployment rates by sex, age and NUTS 2 regi...",lfst_r_lfu3rt,dataset,11.11.2020,08.02.2021,1999,2019
...,...,...,...,...,...,...,...
9567,"Unemployment by sex, age and country of birth ...",lfsq_ugacob,dataset,05.01.2021,08.02.2021,1998Q1,2020Q3
9599,"Unemployment rate by sex, age, migration statu...",lfso_14luner,dataset,28.03.2019,08.02.2021,2014,2014
9664,"Unemployment rates by sex, age and citizenship...",lfsa_urgan,dataset,11.11.2020,08.02.2021,1995,2019
9695,Unemployment by sex and age – annual data,une_rt_a,dataset,07.10.2020,08.02.2021,1992,2019


In [19]:
eurostat.get_data_df('ei_hppi_q', flags=False)

,indic,unit,geo\time,2005Q1,2005Q2,2005Q3,2005Q4,2006Q1,2006Q2,2006Q3,...,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,2020Q2,2020Q3
0,TOTAL,I15_NSA,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,118.51,120.37,122.18,122.38,126.40,127.81,129.81,131.85,135.02,139.23
1,TOTAL,I15_NSA,BE,69.68,71.24,73.78,75.24,76.70,78.79,81.07,...,108.73,110.93,110.34,111.54,112.01,115.96,115.61,115.49,117.04,119.56
2,TOTAL,I15_NSA,BG,76.01,79.26,81.55,83.52,87.47,88.91,92.96,...,124.03,124.99,126.15,129.45,130.26,131.97,133.99,135.54,134.07,138.33
3,TOTAL,I15_NSA,CY,93.41,95.38,98.79,100.64,103.04,106.44,110.64,...,104.01,103.12,107.04,107.93,112.73,105.64,106.51,109.13,109.48,104.21
4,TOTAL,I15_NSA,CZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,128.70,131.90,134.40,137.20,140.60,143.40,146.40,149.00,151.50,155.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,TOTAL,RT4,SE,NaN,NaN,NaN,NaN,14.00,13.50,12.10,...,-1.70,-2.10,0.70,1.50,2.20,2.90,3.30,4.50,3.30,3.70
103,TOTAL,RT4,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.70,9.40,8.30,7.60,7.40,7.10,4.80,4.70,5.20,3.30
104,TOTAL,RT4,SK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.00,4.40,6.80,5.70,8.30,11.50,10.90,13.10,9.70,8.50
105,TOTAL,RT4,TR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.80,6.30,4.50,3.20,1.70,6.80,9.90,15.00,25.70,27.40


In [20]:
# get all dimension of the dataset
dims = eurostat.get_sdmx_dims('ei_hppi_q')
dims

['FREQ', 'GEO', 'INDIC', 'INDICATORS', 'OBS_FLAG', 'TIME', 'UNIT']

In [21]:
# get all possible value of one dimension of the dataset
dic = eurostat.get_sdmx_dic('ei_hppi_q', 'GEO')
dic

{'AT': 'Austria',
 'BE': 'Belgium',
 'BG': 'Bulgaria',
 'CH': 'Switzerland',
 'CY': 'Cyprus',
 'CZ': 'Czechia',
 'DE': 'Germany (until 1990 former territory of the FRG)',
 'DK': 'Denmark',
 'EA': 'Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015)',
 'EA19': 'Euro area - 19 countries  (from 2015)',
 'EE': 'Estonia',
 'ES': 'Spain',
 'EU': 'European Union (EU6-1958, EU9-1973, EU10-1981, EU12-1986, EU15-1995, EU25-2004, EU27-2007, EU28-2013, EU27-2020)',
 'EU27_2020': 'European Union - 27 countries (from 2020)',
 'EU28': 'European Union - 28 countries (2013-2020)',
 'FI': 'Finland',
 'FR': 'France',
 'HR': 'Croatia',
 'HU': 'Hungary',
 'IE': 'Ireland',
 'IS': 'Iceland',
 'IT': 'Italy',
 'LT': 'Lithuania',
 'LU': 'Luxembourg',
 'LV': 'Latvia',
 'MT': 'Malta',
 'NL': 'Netherlands',
 'NO': 'Norway',
 'PL': 'Poland',
 'PT': 'Portugal',
 'RO': 'Romania',
 'SE': 'Sweden',
 'SI': 'Slovenia',
 'SK': 'Slovakia',
 'TR': 'Turkey',
 'UK': 'United Kingd

In [22]:
dic = eurostat.get_sdmx_dic('ei_hppi_q', 'INDIC')
dic

{'TOTAL': 'Total'}

In [23]:
dic = eurostat.get_sdmx_dic('ei_hppi_q', 'UNIT')
dic

{'I15_NSA': 'Index, 2015=100 (NSA)',
 'RT1': 'Growth rate on previous period (t/t-1)',
 'RT4': 'Growth rate on the same quarter in previous year'}

In [24]:
# proxyinfo = {'http': ['myuser', 'mypassword', '123.456.789.012:8012'],
#                 'https': ['myuser', 'mypassword', 'url:port']}
# setproxy(proxyinfo)

StartPeriod = 2019
EndPeriod = 2020

filter_pars = { 'INDIC': ['TOTAL',], 
              'UNIT': ['I15_NSA','RT1', 'RT4'],
              'GEO':['LU',]
             }

eurostat.get_sdmx_data_df('ei_hppi_a', StartPeriod, EndPeriod, filter_pars, flags = True, verbose=True)


Server Error 500: ei_hppi_a not found in the Eurostat server


SDMXException: ('Server error:', 500, 'http://ec.europa.eu/eurostat/SDMX/diss-web/rest/dataflow/ESTAT/ei_hppi_a/latest?references=all')

In [ ]:
StartPeriod = 2007
EndPeriod = 2008
filter_pars = {'FREQ': ['A',], 'PRCCODE': ['08111250','08111150']}
df = eurostat.get_sdmx_data_df('DS-066341', StartPeriod, EndPeriod, filter_pars, flags = True, verbose=True)
df

In [ ]:
#----Unemployment Rate---------
dataEU_unempl=pd.read_json('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/ei_lmhr_m?geo=EA&indic=LM-UN-T-TOT&s_adj=NSA&unit=PC_ACT',typ='series',orient='table',numpy=True) #,typ='DataFrame',orient='table'
x=[]
for i in range(int(sorted(dataEU_unempl['value'].keys())[0]),1+int(sorted(dataEU_unempl['value'].keys(),reverse=True)[0])):
    x=numpy.append(x,dataEU_unempl['value'][str(i)])
EU_unempl=pd.Series(x,index=pd.date_range((pd.to_datetime((sorted(dataEU_unempl['dimension']['time']['category']['index'].keys())[(sorted(int(v) for v in dataEU_unempl['value'].keys())[0])]),format='%YM%M')), periods=len(x), freq='M')) #'1/1993'


#----GDP---------
dataEU_GDP=pd.read_json('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/namq_10_gdp?geo=EA&na_item=B1GQ&s_adj=NSA&unit=CP_MEUR',typ='series',orient='table',numpy=True) #,typ='DataFrame',orient='table'
x=[]
for i in range((sorted(int(v) for v in dataEU_GDP['value'].keys())[0]),1+(sorted((int(v) for v in dataEU_GDP['value'].keys()),reverse=True))[0]):
    x=numpy.append(x,dataEU_GDP['value'][str(i)])
EU_GDP=pd.Series(x,index=pd.date_range((pd.Timestamp(sorted(dataEU_GDP['dimension']['time']['category']['index'].keys())[(sorted(int(v) for v in dataEU_GDP['value'].keys())[0])])), periods=len(x), freq='Q'))


#----Money market interest rates---------
dataEU_intRates=pd.read_json('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/irt_st_m?geo=EA&intrt=MAT_ON',typ='series',orient='table',numpy=True) #,typ='DataFrame',orient='table'
x=[]
for i in range((sorted(int(v) for v in dataEU_intRates['value'].keys())[0]),1+(sorted((int(v) for v in dataEU_intRates['value'].keys()),reverse=True))[0]):
    x=numpy.append(x,dataEU_intRates['value'][str(i)])
EU_intRates=pd.Series(x,index=pd.date_range((pd.to_datetime((sorted(dataEU_intRates['dimension']['time']['category']['index'].keys())[(sorted(int(v) for v in dataEU_intRates['value'].keys())[0])]),format='%YM%M')), periods=len(x), freq='M'))
﻿

## unemployment rate

In [ ]:
dic = eurostat.get_sdmx_dic('irt_st_m', 'GEO')
dic

In [ ]:
StartPeriod = 2019
EndPeriod = 2020

filter_pars = { 
              'GEO':['EA',]
             }

eurostat.get_sdmx_data_df('irt_st_a', StartPeriod, EndPeriod, filter_pars, flags = True, verbose=True)


In [ ]:
eurostat.get_data_df('irt_st_m', flags=False)

In [ ]:
StartPeriod = '2019-01-01'
EndPeriod = '2021-01-01'

filter_pars = { 
              'GEO':['EA']
             }

eurostat.get_sdmx_data_df('irt_st_a', StartPeriod, EndPeriod, filter_pars, flags = True, verbose=True)


In [ ]:
df = eurostat.get_data_df('irt_st_q', flags=False)